In [1]:
# Step 1: Import Libraries
import torch
import torch.nn as nn
import numpy as np

# Step 2: Sample Text Data (can be replaced with any large corpus)
text = "hello world handwritten text generation using rnn model in pytorch"
chars = sorted(list(set(text)))
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}
n_chars = len(chars)

# Step 3: One-Hot Encoding Function
def one_hot_encode(sequence, n_chars):
    data = np.zeros((len(sequence), n_chars))
    for i, char in enumerate(sequence):
        data[i, char_to_idx[char]] = 1
    return data

# Step 4: Create Input and Target Sequences
seq_length = 10
input_data = []
target_data = []

for i in range(0, len(text) - seq_length):
    input_seq = text[i:i+seq_length]
    target_char = text[i+seq_length]
    input_data.append(one_hot_encode(input_seq, n_chars))
    target_data.append(char_to_idx[target_char])

# ⚠️ Performance Fix: Convert list of arrays to a single NumPy array first
input_array = np.array(input_data)
X = torch.tensor(input_array, dtype=torch.float32)       # shape: (samples, seq_length, n_chars)
y = torch.tensor(target_data, dtype=torch.long)          # shape: (samples)

# Step 5: Define the RNN Model
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.rnn = nn.LSTM(input_size, hidden_size, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out[:, -1, :])  # Take only the last output
        return out, hidden

    def init_hidden(self, batch_size):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size),
                torch.zeros(self.n_layers, batch_size, self.hidden_size))

# Step 6: Initialize Model, Loss Function, and Optimizer
model = CharRNN(input_size=n_chars, hidden_size=128, output_size=n_chars, n_layers=1)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

# Step 7: Training the Model
n_epochs = 100
for epoch in range(n_epochs):
    hidden = model.init_hidden(X.size(0))
    optimizer.zero_grad()
    output, hidden = model(X, hidden)
    loss = loss_fn(output, y)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.4f}")

# Step 8: Generate Text Function
def generate_text(model, start_str, gen_length):
    model.eval()
    result = start_str
    hidden = model.init_hidden(1)

    # Prepare initial input
    input_seq = one_hot_encode(start_str[-seq_length:], n_chars)
    input_seq = torch.tensor(input_seq, dtype=torch.float32).unsqueeze(0)

    for _ in range(gen_length):
        output, hidden = model(input_seq, hidden)
        predicted_idx = torch.argmax(output, dim=1).item()
        predicted_char = idx_to_char[predicted_idx]
        result += predicted_char

        # Prepare next input
        next_input = one_hot_encode(result[-seq_length:], n_chars)
        input_seq = torch.tensor(next_input, dtype=torch.float32).unsqueeze(0)

    return result

# Step 9: Test the Generator
generated = generate_text(model, start_str="hello wor", gen_length=100)
print("\nGenerated Text:\n" + generated)


Epoch 10/100, Loss: 2.7298
Epoch 20/100, Loss: 2.5849
Epoch 30/100, Loss: 2.2432
Epoch 40/100, Loss: 1.2910
Epoch 50/100, Loss: 0.3317
Epoch 60/100, Loss: 0.0509
Epoch 70/100, Loss: 0.0134
Epoch 80/100, Loss: 0.0061
Epoch 90/100, Loss: 0.0038
Epoch 100/100, Loss: 0.0028

Generated Text:
hello wordd d nndit nttexxgmon gmod d yrceetxxgmletexggmd prhd pode  yornd ai ptchnnohin toxngmhinnpohinpochi
